In [19]:
import csv, random, numpy as np
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Cropping2D
from keras.preprocessing.image import img_to_array, load_img, flip_axis, random_shift
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2, activity_l2

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import itertools

import cv2
from PIL import Image

In [22]:
def model(load, shape, checkpoint=None):
    """Return a model from file or to train on."""
    if load and checkpoint: return load_model(checkpoint)

#    conv_layers1, conv_layers2, dense_layers = [24, 36, 48], [64, 64], [1024, 512]
    dense_layers = [1024, 512]
    
    model = Sequential()
    
    model.add(Cropping2D(cropping=((40,20), (0,0)), input_shape=shape))
    model.add(Lambda(lambda x: x / 255.0 - 0.5))

    model.add(Convolution2D(24,5, 5, subsample=(2, 2), W_regularizer=l2(0.001)))
    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
    
    
    model.add(Convolution2D(36,5, 5, subsample=(2, 2), W_regularizer=l2(0.001), activation='elu'))
#    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
   

    model.add(Convolution2D(48,5, 5, subsample=(2, 2), W_regularizer=l2(0.001), activation='elu'))
#    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))


    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), W_regularizer=l2(0.001), activation='elu'))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
    
    
    model.add(Flatten())
    for dl in dense_layers:
        model.add(Dense(dl,W_regularizer=l2(0.001), activation='elu'))
#        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer="adam")
    return model

In [5]:
def get_X_y(data_file, flag='unix'):
    """Read the log file and turn it into X/y pairs. Add an offset to left images, remove from right images."""
    X, y = [], []
#    images = []
    steering_offset = 0.4
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):
                     
            if(flag == 'window' and float(speed) > 3):
                
#                if float(speed) < 3: continue  # throw away low-speed samples
                
                for copy in range(30):
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]
                    
                        
                    if(random.random() < 1.0):
                        X += [left_img.strip(), right_img.strip()]
                        y += [float(steering_angle) + steering_offset, float(steering_angle) - steering_offset]
            
                    else:    
                        X += [left_img.strip(), right_img.strip(),center_img.strip()]
                        y += [float(steering_angle) + steering_offset, float(steering_angle) - steering_offset,float(steering_angle)]
                    
            if(flag == 'linux' and float(speed) > 10):
                          
 #               if float(speed) < 10: continue  # throw away low-speed samples
            
                for copy in range(1):
                    left_img = left_img.split('/')[-1]
                    right_img = right_img.split('/')[-1]
                    center_img = center_img.split('/')[-1]
                    
                    
                    if(random.random() < 0.5):
                        X += [left_img.strip(), right_img.strip()]
                        y += [float(steering_angle) + steering_offset, float(steering_angle) - steering_offset]
            
                    else:    
                        X += [left_img.strip(), right_img.strip(),center_img.strip()]
                        y += [float(steering_angle) + steering_offset, float(steering_angle) - steering_offset,float(steering_angle)]
                     
            
    return X, y

In [6]:
def random_darken(image):
    """Given an image (from Image.open), randomly darken a part of it."""
    w, h = image.size

    # Make a random box.
    x1, y1 = random.randint(0, w), random.randint(0, h)
    x2, y2 = random.randint(x1, w), random.randint(y1, h)

    # Loop through every pixel of our box (*GASP*) and darken.
    for i in range(x1, x2):
        for j in range(y1, y2):
            new_value = tuple([int(x * 0.5) for x in image.getpixel((i, j))])
            image.putpixel((i, j), new_value)
    return image

In [7]:
data_dir1 = ('../data2/driving_log.csv','linux')
data_dir2 = ('../20170320/driving_log.csv','window')

print(type(data_dir1))
print(data_dir1[0])

def merge_data(dir1, dir2):
    
    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1])
    
    X = X2 + X1
    y = y2 + y1
    
    return X,y

X_merge, y_merge = merge_data(data_dir1, data_dir2) 

print(len(X_merge))
print(len(y_merge))
print(X_merge[0])
print(y_merge[0])

<class 'tuple'>
../data2/driving_log.csv
31036
31036
left_2017_03_20_20_02_42_608.jpg
0.4


In [8]:
def split_data(data, label, split = 0.3):
    
    train_samples, validation_samples, train_labels, validation_labels = train_test_split(
    data,
    label,
    test_size=split,
    random_state=832289)
    
    return train_samples, validation_samples, train_labels, validation_labels

X_merge,y_merge = shuffle(X_merge, y_merge)
train_samples, validation_samples, train_labels, validation_labels = split_data(X_merge,y_merge)

print(len(train_samples))
print(len(validation_samples))
print(len(train_labels))
print(len(validation_labels))
print(train_samples[0])
print(validation_samples[0])
print(train_labels[0])
print(validation_labels[0])

21725
9311
21725
9311
left_2017_03_20_20_03_17_386.jpg
left_2017_03_20_20_02_54_648.jpg
0.4
0.4


In [9]:
def process_image_test(image, steering_angle, augment=True, shape=(160,320)):
    """Process and augment an image."""

    pil_image = Image.fromarray(image)

    if augment and random.random() < 0.5:
        pil_image = random_darken(pil_image)  # before numpy'd

    image = img_to_array(pil_image)
        
    if augment:
        image = random_shift(image, 0, 0.2, 0, 1, 2)  # only vertical
        if random.random() < 0.5:
            image = flip_axis(image, 1)
            steering_angle = -steering_angle

    return image, steering_angle

In [10]:
def generator(samples, labels, batch_size=128):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples, labels)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            images = []
            angles = []
            for idx, val in enumerate(batch_samples):
#                name = './IMG/'+batch_sample[0].split('/')[-1]
        
                name = batch_samples[idx]
                
                path = '../merge_image/image/' + name 
            
                raw_image = cv2.imread(path)
                raw_angle = float(batch_labels[idx])
                
                raw_image, raw_angle = process_image_test(raw_image, raw_angle)
                
                images.append(raw_image)
                angles.append(raw_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)
            
def generator_valid(samples, labels, batch_size=128):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples, labels)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            images = []
            angles = []
            for idx, val in enumerate(batch_samples):
#                name = './IMG/'+batch_sample[0].split('/')[-1]
        
                name = batch_samples[idx]
                
                path = '../merge_image/image/' + name 
            
                raw_image = cv2.imread(path)
                raw_angle = float(batch_labels[idx])
                
#                raw_image, raw_angle = process_image_test(raw_image, raw_angle)
                
                images.append(raw_image)
                angles.append(raw_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples,train_labels, batch_size=128)
validation_generator = generator_valid(validation_samples, validation_labels, batch_size=128)

In [28]:
def train_valid():
    """Load our network and our data, fit the model, save it."""
    net = model(load=False, shape=(160, 320, 3))
    
    
    X_merge, y_merge = merge_data(data_dir1, data_dir2)
    
    print(len(X_merge))
    
    X_merge,y_merge = shuffle(X_merge, y_merge)
    
    train_samples, validation_samples, train_labels, validation_labels = split_data(X_merge,y_merge,split = 0.1)
    
    train_generator = generator(train_samples, train_labels, batch_size=256)
    validation_generator = generator_valid(validation_samples, validation_labels, batch_size=256)

    
    net.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, 
            nb_val_samples=len(validation_samples), nb_epoch=10)
    net.save('model01.h5')

In [ ]:
train_valid()

31084
Epoch 1/10
27975/27975 [==============================] - 275s - loss: 1.2943 - val_loss: 0.5843

In [13]:
print(type(X_merge))

<class 'list'>
